# Additional Cython Features

## Automatic Type Inference Using Cython's `infer_types`

In [1]:
import numpy as np
from random import random
import Cython

%load_ext Cython

In [2]:
%%cython -a

from random import random
from cython cimport infer_types

cdef inline double my_rand():
    return random()

@infer_types(True)       # automatically infers the type of the data 
cpdef pi_mc_inferred(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x = my_rand()
        y = my_rand()
        if x * x + y * y <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [3]:
%time pi_mc_inferred(10000)

CPU times: user 430 µs, sys: 228 µs, total: 658 µs
Wall time: 661 µs


3.1608

## Cython Extensions Types

In [4]:
class PyRectangle:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def area(self):
        return self.x * self.y
    
    def perimeter(self):
        return 2.0 * (self.x + self.y)

In [5]:
%%cython 

cdef class CyRectangle:
    cdef:
        double x, y
        
    def __cinit__(self, x, y):
        self.x = x
        self.y = y
        
    cpdef double area(self):
        return self.x * self.y
    
    cpdef double perimeter(self):
        return 2.0 * (self.x + self.y)

In [16]:
a = CyRectangle(1, 2)
print(a.area(), a.perimeter())

2.0 6.0


In [17]:
%%cython
from random import random

cdef class CyRectangle:
    cdef:
        double x, y
        
    def __cinit__(self, x, y):
        self.x = x
        self.y = y
        
    cpdef double area(self):
        return self.x * self.y
    
    cpdef double perimeter(self):
        return 2.0 * (self.x + self.y)
    
cdef class CyRectangles:
    cdef:
        list rectangles
        
    def __cinit__(self, int n):
        cdef unsigned int i
        self.rectangles = []
        for i in range(n):
            self.rectangles.append(CyRectangle(random(), random()))
            
    cpdef double total_area(self):
        cdef CyRectangle rect
        cdef double area = 0.0
        for rect in self.rectangles:
            area += rect.area()
            
        return area

In [18]:
a = CyRectangles(100000)

In [19]:
a.total_area()

25010.549315739565

## C-like Allocation/Dealllocation

In [20]:
%%cython

from libc.stdlib cimport malloc, free


cdef class CyRangeVector:
    cdef:
        int *data
        int size
        
    def __cinit__(self, int start, int end):
        cdef unsigned int i
        if start >= end:
            raise Exception(f'{start} >= {end}')
        self.size = end - start
        self.data = <int*>malloc(self.size * sizeof(int))  # cast the results form `malloc` into an int-type pointer (CYTHON CASTING SYNTAX)
        
        for i in range(start, end):
            self.data[i - start] = i
            
    def __getitem__(self, int i):
        if i >= self.size or i < 0:
            return -1

        return self.data[i]
    
    def __dealloc__(self):
        free(self.data)      # important for garbage collecting (which python does not know how to do from `malloc` allocation)

/users/class452/.cache/ipython/cython/_cython_magic_7324501553d686cf0a3b3ce7661f20a2.c: In function '__pyx_pf_46_cython_magic_7324501553d686cf0a3b3ce7661f20a2_13CyRangeVector___cinit__':
/users/class452/.cache/ipython/cython/_cython_magic_7324501553d686cf0a3b3ce7661f20a2.c:1536:45: warning: comparison of integer expressions of different signedness: 'unsigned int' and 'int' [-Wsign-compare]
 1536 |   for (__pyx_t_8 = __pyx_v_start; __pyx_t_8 < __pyx_t_7; __pyx_t_8+=1) {
      |                                             ^


In [24]:
my_range = CyRangeVector(10, 11000)
my_range[2]

12

## Interacting with the C++ Standard Template Library

As long as we start using the C++ STL from inside Cython we have to switch to `language=c++`

In [ ]:
%%cython

# distutils: language=c++

from libcpp.vector cimport vector

cdef class CyRangeVector:
    cdef:
        vector[int] data      # `vector` construct from C++ automatically allocates/deallocates memory
        
    def __cinit__(self, int start, int end):
        cdef unsigned int i
        if start >= end:
            raise Exception(f'{start} >= {end}')
        for i in range(start, end):
            self.data.push_back(i)
            
    def __getitem__(self, int i):
        if i >= self.data.size() or i < 0:
            return None
        
        return self.data[i]

In [ ]:
v = CyRangeVector(1, 20)
print(v[1])

In [ ]:
%%cython

# distutils: language=c++

from libcpp.vector cimport vector

cpdef vector[int] cy_range(int start, int end):
    cdef vector[int] v
    cdef unsigned int i
    for i in range(start, end):
        v.push_back(i)
    
    return v

In [ ]:
x = cy_range(1, 10)
print(x, type(x))

### Example of interacting with the `random` STL library (Cython 3)

In [ ]:
%%cython

#distutils: language=c++
#distutils: extra_compile_args = -std=c++11
#distutils: extra_link_args = -std=c++11


from libcpp.random cimport random_device, mt19937, uniform_real_distribution

cdef class my_uniform:
    cdef:
        mt19937 mt
        uniform_real_distribution[double] uni
        
    def __cinit__(self, ):
        cdef random_device rd
        self.mt = mt19937(rd())
        self.uni = uniform_real_distribution[double](0.0, 1.0)
        
    def rand_uni(self):
        return self.uni(self.mt)

In [ ]:
x = my_uniform()

In [ ]:
%timeit x.rand_uni()

In [ ]:
%%cython -a

#distutils: language=c++
#distutils: extra_compile_args = -std=c++11
#distutils: extra_link_args = -std=c++11

from cython cimport infer_types

from libcpp.random cimport random_device, mt19937, uniform_real_distribution
from libcpp.pair cimport pair

cdef class my_uniform:
    cdef:
        mt19937 mt
        uniform_real_distribution[double] uni
        
    def __cinit__(self, ):
        cdef random_device rd
        self.mt = mt19937(rd())
        self.uni = uniform_real_distribution[double](0.0, 1.0)
        
    cdef rand_uni(self):
        return pair[double, double](self.uni(self.mt), self.uni(self.mt))
    
@infer_types(True)
cpdef pi_mc_random(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    my_uni = my_uniform()
    for i in range(n):
        x, y = my_uni.rand_uni()
        if x * x + y * y <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
pi_mc_random(100000)

### Additional STL libraries for Cython 0.29.x are available and you can look at their [definition files](https://github.com/cython/cython/tree/0.29.x/Cython/Includes/libcpp)

### More STL libraries are available for Cython 3 [here](https://github.com/cython/cython/tree/master/Cython/Includes/libcpp)